In [12]:
import numpy as np
from sklearn import tree,metrics,svm,ensemble,neighbors
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import xgboost as xgb
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline

In [13]:
mixture = ['氮化钒铁FeV55N11-A','低铝硅铁','钒氮合金(进口)','钒铁(FeV50-A)','钒铁(FeV50-B)','硅铝钙','硅铝合金FeAl30Si25','硅铝锰合金球',
           '硅锰面（硅锰渣）','硅铁(合格块)','硅铁FeSi75-B','石油焦增碳剂','锰硅合金FeMn64Si27(合格块)','锰硅合金FeMn68Si18(合格块)',
           '碳化硅(55%)','硅钙碳脱氧剂']
price = [350000,6500,350000,205000,205000,11800,1000,8500,7600,6000,6000,4600,8150,8150,6100,4000]
len(price)

16

In [38]:
data = pd.read_excel('./data1.xlsx')
data2 = pd.read_excel('data2.xlsx')
# data[mixture].head(1)
data2.head(2)
data.isna().sum()

炉号                        0
钢种                        0
钢号                        0
转炉终点温度                    1
转炉终点C                     1
转炉终点Mn                 1466
转炉终点S                     1
转炉终点P                  1466
转炉终点Si                    1
钢水净重                      2
连铸正样C                   906
连铸正样Mn                  906
连铸正样S                   906
连铸正样P                   906
连铸正样Si                  906
连铸正样Ceq_val              11
连铸正样Cr                  906
连铸正样Ni_val                2
连铸正样Cu_val                2
连铸正样Nb_val             1392
连铸正样V_val                 2
连铸正样Alt_val               2
连铸正样Als_val               2
连铸正样Mo_val               17
连铸正样Ti_val               56
连铸正样N_val              1716
连铸正样O_val              1716
连铸正样B_val              1578
氮化钒铁FeV55N11-A            0
低铝硅铁                      0
钒氮合金(进口)                  0
钒铁(FeV50-A)               0
钒铁(FeV50-B)               0
钒铁(FeV50-B).1             0
硅铝钙                       0
硅铝合金FeAl30Si25      

In [ ]:
MnTrain = pd.read_excel('data1.xlsx')[nume]
# Mn初始含量较低，对反应收得率影响小，使用均值替换缺失值
MnTrain[['转炉终点Mn']] = MnTrain[['转炉终点Mn']].fillna(MnTrain[['转炉终点Mn']].mean())
# 去掉为对合金化后合金钢采样的数据
MnTrain = MnTrain.drop(MnTrain[MnTrain['连铸正样Mn'].isnull()].index)
# 使用均值替代少量的缺失值
MnTrain[nume] = MnTrain[nume].fillna(MnTrain[nume].mean())
# 重新计算Mn收得率
mn_t=['硅铝锰合金球','硅锰面（硅锰渣）','锰硅合金FeMn64Si27(合格块)','锰硅合金FeMn68Si18(合格块)']
mn_p = [0.3,0.664,0.664,0.664]
mn_total=(MnTrain[mn_t]*mn_p).sum(axis=1)
mn_comsu=(MnTrain['连铸正样Mn']-MnTrain['转炉终点Mn'])*MnTrain['钢水净重']
MnTrain['Mn收得率']=mn_comsu/(mn_total)
# 删掉异常数据
MnTrain = MnTrain.drop(MnTrain[MnTrain['Mn收得率']>1.5].index)
# 构造训练数据并对自变量值进行归一化
X = MnTrain[nume].apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)+1e-9))
Y = MnTrain[['Mn收得率']]
train_data, test_data, train_target, test_target = train_test_split(X.values,Y.values,test_size=0.2)
len(MnTrain.values)

model = Pipeline([('poly', PolynomialFeatures(degree=2)),('linear', linear_model.LinearRegression(fit_intercept=False))])
clf = model.fit(train_data,train_target.flatten().astype(np.float32))
pre_y = clf.predict(test_data)
metrics.r2_score(pre_y,test_target),metrics.mean_absolute_error(pre_y,test_target),metrics.mean_squared_error(pre_y,test_target)

In [37]:
data.corr()[['连铸正样P']].sort_values('连铸正样P',ascending=False)

,连铸正样P
连铸正样P,1.000000
转炉终点P,0.624525
转炉终点Mn,0.346486
连铸正样Cr,0.330648
连铸正样S,0.125186
转炉终点温度,0.100989
硅铝钙,0.098213
连铸正样Mn,0.093182
石油焦增碳剂,0.062622
碳化硅(55%),0.058972


### 检验初始状态的P,Mn对

### 数据1缺失值处理

In [19]:
help(data.drop)

Help on method drop in module pandas.core.frame:

drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise') method of pandas.core.frame.DataFrame instance
    Drop specified labels from rows or columns.
    
    Remove rows or columns by specifying label names and corresponding
    axis, or by specifying directly index or column names. When using a
    multi-index, labels on different levels can be removed by specifying
    the level.
    
    Parameters
    ----------
    labels : single label or list-like
        Index or column labels to drop.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Whether to drop labels from the index (0 or 'index') or
        columns (1 or 'columns').
    index, columns : single label or list-like
        Alternative to specifying axis (``labels, axis=1``
        is equivalent to ``columns=labels``).
    
        .. versionadded:: 0.21.0
    level : int or level name, optional
        For MultiIndex, level

In [26]:
# 去掉初始Mn\P
try:
    data.drop(labels=['转炉终点Mn','转炉终点P','连铸正样Nb_val','连铸正样N_val','连铸正样O_val','连铸正样B_val'],axis=1,inplace=True)
    
except:
    print('err')

err


In [18]:
data.isna().sum()

炉号                        0
钢种                        0
钢号                        0
转炉终点温度                    1
转炉终点C                     1
转炉终点Mn                 1466
转炉终点S                     1
转炉终点P                  1466
转炉终点Si                    1
钢水净重                      2
连铸正样C                   906
连铸正样Mn                  906
连铸正样S                   906
连铸正样P                   906
连铸正样Si                  906
连铸正样Ceq_val              11
连铸正样Cr                  906
连铸正样Ni_val                2
连铸正样Cu_val                2
连铸正样Nb_val             1392
连铸正样V_val                 2
连铸正样Alt_val               2
连铸正样Als_val               2
连铸正样Mo_val               17
连铸正样Ti_val               56
连铸正样N_val              1716
连铸正样O_val              1716
连铸正样B_val              1578
氮化钒铁FeV55N11-A            0
低铝硅铁                      0
钒氮合金(进口)                  0
钒铁(FeV50-A)               0
钒铁(FeV50-B)               0
钒铁(FeV50-B).1             0
硅铝钙                       0
硅铝合金FeAl30Si25      

## HRB400B

,合金配料,C,Mn,S,P,Si,Al,Ca,V,N,价格(元/吨)
0,氮化钒铁FeV55N11-A,0.000000,0.000,0.0000,0.0000,0.000,0.0000,0.000000,0.560,0.119,350000
1,低铝硅铁,0.000000,0.000,0.0000,0.0000,0.740,0.0008,0.000000,0.000,0.000,6500
2,钒氮合金(进口),0.000000,0.000,0.0000,0.0000,0.000,0.0000,0.000000,0.780,0.151,350000
3,钒铁(FeV50-A),0.003100,0.000,0.0002,0.0006,0.012,0.0000,0.000000,0.512,0.000,205000
4,钒铁(FeV50-B),0.003100,0.000,0.0002,0.0006,0.012,0.0000,0.000000,0.512,0.000,205000
5,硅铝钙,0.000000,0.000,0.0000,0.0000,0.341,0.0910,0.148000,0.000,0.000,11800
6,硅铝合金FeAl30Si25,0.003740,0.000,0.0000,0.0000,0.285,0.3170,0.000000,0.000,0.000,1000
7,硅铝锰合金球,0.000000,0.300,0.0000,0.0000,0.300,0.4000,0.000000,0.000,0.000,8500
8,硅锰面（硅锰渣）,0.017000,0.664,0.0002,0.0018,0.072,0.0000,0.000000,0.000,0.000,7600
9,硅铁(合格块),0.000600,0.000,0.0002,0.0004,0.767,0.0000,0.000000,0.000,0.000,6000
